In [83]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [84]:
PATH = os.getcwd()

In [85]:
train_dataset = image_dataset_from_directory(
                                             batch_size=64,
                                             validation_split=0.2,
                                             directory=os.path.join(PATH, 'kaggle/working/train/'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(230, 224),
                                             subset="training",
                                             seed=0)
train_dataset

Found 10 files belonging to 1 classes.
Using 8 files for training.


<BatchDataset shapes: ((None, 230, 224, 3), (None,)), types: (tf.float32, tf.int32)>

In [86]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=64,
                                             validation_split=0.2,
                                             directory=os.path.join(PATH, 'kaggle/working/test/'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(230, 224),
                                             subset="training",
                                             seed=0)

Found 10 files belonging to 1 classes.
Using 8 files for training.


In [88]:
def prepare(ds, augment=False):
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])
    # Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False).batch(64)
test_dataset = prepare(test_dataset, augment=False).batch(64)
train_dataset

<MapDataset shapes: ((None, None, 230, 224, 3), (None, None)), types: (tf.float32, tf.int32)>

In [90]:
model = models.Sequential([
    layers.Input(shape=(230, 224, 3)),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Conv2D(100, 3, activation='relu'),
    layers.Dropout(0.25),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='softmax'),
])

model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 228, 222, 128)     3584      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 226, 220, 100)     115300    
_________________________________________________________________
dropout_30 (Dropout)         (None, 226, 220, 100)     0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 226, 220, 100)     400       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 113, 110, 100)     0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 113, 110, 100)     0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 113, 110, 100)   

In [92]:
model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [93]:
history = model.fit(train_dataset,
                   validation_data=test_dataset,
                   epochs=10,
                   batch_size=5)

Epoch 1/10


ValueError: in user code:

    /home/bishwas76/Documents/Major_Project/env/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/bishwas76/Documents/Major_Project/env/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/bishwas76/Documents/Major_Project/env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/bishwas76/Documents/Major_Project/env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/bishwas76/Documents/Major_Project/env/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replic